In [1]:
!pip install -q emoji
!pip install -q vncorenlp
!pip install -q datasets
!pip install -q transformers
!pip install -q py_vncorenlp

In [16]:
import os
import numpy as np
import pandas as pd

import emoji
import vncorenlp
import datasets
import transformers
import py_vncorenlp
import regex as re
from py_vncorenlp import VnCoreNLP

In [18]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk-headless -qq > /dev/null

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]    
Get:3 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Get:4 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3071 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1470 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [33.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1512 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3867 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4218 kB]
Get:13 http://security.ubuntu.com/ubuntu 

In [19]:
py_vncorenlp.download_model(save_dir='/kaggle/working')
annotator = VnCoreNLP(save_dir='/kaggle/working')

VnCoreNLP model folder /kaggle/working already exists! Please load VnCoreNLP from this folder!
2024-06-27 03:39:41 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-06-27 03:39:41 INFO  PosTagger:23 - Loading POS Tagging model
2024-06-27 03:39:45 INFO  NerRecognizer:34 - Loading NER model
2024-06-27 03:40:04 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [10]:
VN_CHARS = 'áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ'

VOWELS_TABLE = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']
]


VOWELS_TO_IDS = {
    'a': (0, 0), 'à': (0, 1), 'á': (0, 2), 'ả': (0, 3), 'ã': (0, 4), 'ạ': (0, 5), 
    'ă': (1, 0), 'ằ': (1, 1), 'ắ': (1, 2), 'ẳ': (1, 3), 'ẵ': (1, 4), 'ặ': (1, 5), 
    'â': (2, 0), 'ầ': (2, 1), 'ấ': (2, 2), 'ẩ': (2, 3), 'ẫ': (2, 4), 'ậ': (2, 5), 
    'e': (3, 0), 'è': (3, 1), 'é': (3, 2), 'ẻ': (3, 3), 'ẽ': (3, 4), 'ẹ': (3, 5), 
    'ê': (4, 0), 'ề': (4, 1), 'ế': (4, 2), 'ể': (4, 3), 'ễ': (4, 4), 'ệ': (4, 5), 
    'i': (5, 0), 'ì': (5, 1), 'í': (5, 2), 'ỉ': (5, 3), 'ĩ': (5, 4), 'ị': (5, 5), 
    'o': (6, 0), 'ò': (6, 1), 'ó': (6, 2), 'ỏ': (6, 3), 'õ': (6, 4), 'ọ': (6, 5), 
    'ô': (7, 0), 'ồ': (7, 1), 'ố': (7, 2), 'ổ': (7, 3), 'ỗ': (7, 4), 'ộ': (7, 5), 
    'ơ': (8, 0), 'ờ': (8, 1), 'ớ': (8, 2), 'ở': (8, 3), 'ỡ': (8, 4), 'ợ': (8, 5), 
    'u': (9, 0), 'ù': (9, 1), 'ú': (9, 2), 'ủ': (9, 3), 'ũ': (9, 4), 'ụ': (9, 5), 
    'ư': (10, 0), 'ừ': (10, 1), 'ứ': (10, 2), 'ử': (10, 3), 'ữ': (10, 4), 'ự': (10, 5), 
    'y': (11, 0), 'ỳ': (11, 1), 'ý': (11, 2), 'ỷ': (11, 3), 'ỹ': (11, 4), 'ỵ': (11, 5)
}

VINAI_NORMALIZED_TONE = {
    'òa': 'oà', 'Òa': 'Oà', 'ÒA': 'OÀ', 
    'óa': 'oá', 'Óa': 'Oá', 'ÓA': 'OÁ', 
    'ỏa': 'oả', 'Ỏa': 'Oả', 'ỎA': 'OẢ',
    'õa': 'oã', 'Õa': 'Oã', 'ÕA': 'OÃ',
    'ọa': 'oạ', 'Ọa': 'Oạ', 'ỌA': 'OẠ',
    'òe': 'oè', 'Òe': 'Oè', 'ÒE': 'OÈ',
    'óe': 'oé', 'Óe': 'Oé', 'ÓE': 'OÉ',
    'ỏe': 'oẻ', 'Ỏe': 'Oẻ', 'ỎE': 'OẺ',
    'õe': 'oẽ', 'Õe': 'Oẽ', 'ÕE': 'OẼ',
    'ọe': 'oẹ', 'Ọe': 'Oẹ', 'ỌE': 'OẸ',
    'ùy': 'uỳ', 'Ùy': 'Uỳ', 'ÙY': 'UỲ',
    'úy': 'uý', 'Úy': 'Uý', 'ÚY': 'UÝ',
    'ủy': 'uỷ', 'Ủy': 'Uỷ', 'ỦY': 'UỶ',
    'ũy': 'uỹ', 'Ũy': 'Uỹ', 'ŨY': 'UỸ',
    'ụy': 'uỵ', 'Ụy': 'Uỵ', 'ỤY': 'UỴ',
}

with open('kltn/teencode.txt', 'r', encoding = 'utf8') as f:
    data = f.readlines()
    data = [re.sub(r'[,‘’\';]', '', i).lower() for i in data]
    data = [i.replace('\n', '').replace('\ufeff', '') for i in data]
    slangs_dict = {}
    for i in data:
        j = i.split(':')
        slangs_dict[j[0].strip()] = j[1].strip()

with open('kltn/vietnamese_words.txt', 'r', encoding = 'utf8') as f:
    data = f.readlines()
    vietnamese_words_set = set()
    for i in data:
        vietnamese_words_set.add(i.strip())

In [11]:
def clean_space(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    return text


def remove_html(text):
    return re.sub(r'<[^>]*>', ' ', text)


def remove_emoji(text):
    return emoji.replace_emoji(text, ' ')


def remove_url(text):
    return re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', ' ', text)


def remove_email(text):
    return re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', ' ', text)


def remove_phone_number(text):
    return re.sub(r'^[\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}$', ' ', text)


def remove_unnecessary_characters(text):
    return re.sub(fr"[^\s\w{VN_CHARS}]", ' ', text)


def normalize_unicode(text):
    char1252 = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char_map = dict(zip(char1252.split('|'), charutf8.split('|')))
    return re.sub(char1252, lambda x: char_map[x.group()], text.strip())


def normalize_sentence_typing(text, vinai_normalization=False):
    # https://github.com/VinAIResearch/BARTpho/blob/main/VietnameseToneNormalization.md
    if vinai_normalization: # Just simply replace the wrong tone with the correct one defined by VinAI
        for wrong, correct in VINAI_NORMALIZED_TONE.items():
            text = text.replace(wrong, correct)
        return text.strip()

    # Or you can use this algorithm developed by Behitek to normalize Vietnamese typing in a sentence 
    words = text.strip().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = normalize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


def normalize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars, vowel_indexes = list(word), []
    qu_or_gi, tonal_mark = False, 0

    for index, char in enumerate(chars):
        if char not in VOWELS_TO_IDS: continue
        row, col = VOWELS_TO_IDS[char]
        if index > 0 and (row, chars[index - 1]) in [(9, 'q'), (5, 'g')]:
            chars[index] = VOWELS_TABLE[row][0]
            qu_or_gi = True

        if not qu_or_gi or index != 1: vowel_indexes.append(index)
        if col != 0:
            tonal_mark = col
            chars[index] = VOWELS_TABLE[row][0]

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            index = 1 if len(chars) == 2 else 2
            if chars[index] in VOWELS_TO_IDS:
                row, _ = VOWELS_TO_IDS[chars[index]]
                chars[index] = VOWELS_TABLE[row][tonal_mark]
            else: chars[1] = VOWELS_TABLE[5 if chars[1] == 'i' else 9][tonal_mark]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        row, _ = VOWELS_TO_IDS[chars[index]]
        if row in [4, 8]:
            chars[index] = VOWELS_TABLE[row][tonal_mark]
            return ''.join(chars)

    index = vowel_indexes[0 if len(vowel_indexes) == 2 and vowel_indexes[-1] == len(chars) - 1 else 1] 
    row, _ = VietnameseToneNormalizer.VOWELS_TO_IDS[chars[index]]
    chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][tonal_mark]
    return ''.join(chars)


def is_valid_vietnamese_word(word):
    vowel_indexes = -1 
    for index, char in enumerate(word):
        if char not in VOWELS_TO_IDS: continue
        if vowel_indexes in [-1, index - 1]: vowel_indexes = index
        else: return False
    return True


def fix_slangs(text):
    words = text.split()
    fixed_words = [slangs_dict.get(word.lower(), word) for word in words]
    text = ' '.join(fixed_words)
    return text


def word_processing(text):
    
    # Tách các từ trong chuỗi
    words = text.split()
    
    processed_words = []

    for word in words:
        
        if word.isdigit():
            processed_words.append(word)
            continue
        
        if word.lower() not in vietnamese_words_set:
            # Xác định chữ cái cuối cùng
            last_char = word[-1]

            # Tìm vị trí của ký tự cuối cùng bị lặp
            index = None
            for i in range(len(word)-1, -1, -1):
                if word[i] != last_char:
                    index = i + 1
                    break

            # Loại bỏ các ký tự cuối cùng dư thừa
            if index is not None:
                word = word[:index+1]
            
        # Loại bỏ các từ vô nghĩa
        if word.lower() not in vietnamese_words_set:
            continue

        processed_words.append(word)

    # Tạo chuỗi kết quả
    result = ' '.join(processed_words)   
    return result


# Word segmentation
def word_segmentation(text):
    segmented_text = annotator.word_segment(text)
    return ' '.join(segmented_text)


def text_preprocess(text):
    text = clean_space(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_url(text)
    text = remove_email(text)
    text = remove_phone_number(text)
    text = remove_unnecessary_characters(text)
    text = normalize_unicode(text)
    text = normalize_sentence_typing(text, True)
    text = fix_slangs(text)
    text = word_processing(text)
    text = word_segmentation(text)
    return text